In [3]:
import xarray as xr
import matplotlib.pyplot as plt
import numpy as np
import torch
import os

# List of paths to the .nc files
# '/perm/dadf/HSAF_validation/in_situ_data/pre_processed_data/ismn_nc/soil_TERENO_ISMN_2022.nc'
dir_to_data = "/perm/dadf/HSAF_validation/in_situ_data/pre_processed_data/ismn_nc"

station_name = 'soil_SMOSMANIA_ISMN_2008.nc'

def update_year(file_name, new_year):
    return station_name.replace('2008', str(new_year))

update_year(station_name, '2009')
print(station_name)

nc_files = ['soil_SMOSMANIA_ISMN_2008.nc']
print(nc_files[0])

soil_SMOSMANIA_ISMN_2008.nc
soil_SMOSMANIA_ISMN_2008.nc
